In [1]:
from attr import define, field
from typing import List, Union, ClassVar, Optional, TypeVar
import heapq
from utils import temporary_file
import subprocess
import shlex
from pathlib import Path
import random
from enum import Enum
from utils import END_LINE_CHARS
from pathlib import Path
from sequence import Sequence
from typing import Callable
from copy import deepcopy
import random 
import glob
import json

from typing import TypeVar
import glob
import parse
from datasetconfig import DatasetConfig
from itertools import groupby
from collections import defaultdict
from dataset import Dataset, DatasetType
from labels import BinaryLabel
from seqentry import SeqEntry

In [2]:
U = TypeVar('U')
CHS_ROOT: Path = Path("/home_local/vorontsovie/greco-data/release_7a.2021-10-14/full/CHS/Val_intervals")
BEDTOOLS_PATH: Path = Path("/home_local/dpenzar/bedtools2/bin")

In [3]:
class BedClosestMode(Enum):
    UPSTREAM = "-fu"
    DOWNSTREAM = "-fd"
    ALL = ""

In [4]:
@define
class BedtoolsExecutor:
    bedtools_root: Path

    @property
    def merge_path(self) -> Path:
        return self.bedtools_root / 'mergeBed'

    @property
    def sort_path(self) -> Path:
        return self.bedtools_root / 'sortBed'

    @property
    def subtract_path(self) -> Path:
        return self.bedtools_root / 'subtractBed'

    @property
    def closest_path(self) -> Path:
        return self.bedtools_root / 'closestBed'

    @property
    def flank_path(self) -> Path:
        return self.bedtools_root / 'flankBed'

    @staticmethod
    def run_bedtools_cmd(cmd: str, name: str=""):
        out_path = temporary_file()
        args = shlex.split(cmd)
        with open(out_path, "w") as outp:
            r = subprocess.run(args, stdout=outp, stderr=subprocess.PIPE, text=True)
        if r.stderr:
            raise Exception(f"Bedtools {name} returned error: {r.stderr}")
        return out_path

    def merge(self, path: Path) -> 'Path':
        cmd = f"{self.merge_path} -i {path}" # remove peak columns as ambiguous
        out_path = self.run_bedtools_cmd(cmd, name='merge')
        return out_path

    def subtract(self, a: Path, b: Path, full=False) -> Path:
        cmd = f"{self.subtract_path} {'-A' if full else ''} -a {a} -b {b}"
        out_path = self.run_bedtools_cmd(cmd, name='subtract')
        return out_path

    def closest(self, a: Path, b: Path, how: BedClosestMode) -> Path:
        cmd = f"{self.closest_path} -D ref {how.value} -a {a} -b {b}"
        out_path = self.run_bedtools_cmd(cmd, name='closest')
        return out_path

    def flank(self, path: Path, genomesizes: Path, size: int) -> Path:
        cmd = f"{self.flank_path} -i {path} -g {genomesizes} -b {size}"
        out_path = self.run_bedtools_cmd(cmd, name="flank")
        return out_path

In [5]:
BED_TOOLS_EXECUTOR = BedtoolsExecutor(BEDTOOLS_PATH)

In [6]:
@define
class Genome:
    chroms : dict[str, Sequence]

    def __len__(self)->int:
        return len(self.chroms)

    def __getitem__(self, entry : 'BedEntry') -> Sequence:
        return self.chroms[entry.chr][entry.start:entry.end]

    def cut(self, bed: 'BedData') -> List[Sequence]:
        return [self[e] for e in bed.entries]

    def chrom_sizes(self) -> dict[str, int]:
        return {ch: len(seq.seq) for ch, seq in self.chroms.items()}

    def write_bed_genome_file(self, path: Union[str, Path]):
        if isinstance(path, str):
            path = Path(path)
        ch_sizes = self.chrom_sizes()
        with path.open('w') as outp:
            for ch, size in ch_sizes.items():
                print(ch, size, file=outp, sep="\t")

    @classmethod
    def from_dir(cls, dirpath:Path, ext='.fa'):
        dt = {}
        for chrom in glob.glob(str(dirpath/f"*{ext}")):

            ch_name = Path(chrom).name.replace(ext, "")
            seq = Sequence.from_file(chrom)
            dt[ch_name] = seq

        return cls(dt)

In [7]:
@define(order=True, slots=True)
class BedEntry:
    chr: str
    start: int
    end: int = field(eq=False)
    peak: Optional[int] = field(default=None, eq=False)

    NONE_REPR: ClassVar[str] = '.'
    BED_SEP: ClassVar[str] = "\t"

    @classmethod
    def peak2str(cls, peak: Union[int, None]) -> str:
        if peak is None:
            return cls.NONE_REPR
        else: # isinstance(peak, int):
            return str(peak)
    
    @classmethod
    def str2peak(cls, s: str) -> Union[int, None]:
        if s == cls.NONE_REPR:
            return None
        try:
            return int(s)
        except ValueError:
            pass
        raise Exception("Wrong peak format")

    @classmethod
    def from_line(cls, line: str):
        fields = line.split(cls.BED_SEP)
        if len(fields) < 3:
            raise Exception("Wrong bed format")
        if len(fields) == 3:
            chr, st, end = fields
            return cls(chr, int(st), int(end))
        # len(fields) >= 4
        chr, st, end, peak = fields[0:4]
        return cls(chr, int(st), int(end), cls.str2peak(peak))

    def to_line(self, include_peak: bool=True) -> str:
        fields = [self.chr, str(self.start), str(self.end)]
        if include_peak:
            fields.append(self.peak2str(self.peak))
        return self.BED_SEP.join(fields)

    def __len__(self) -> int:
        return max(0, self.end - self.start) 

    def __getitem__(self, ind: int) -> int:
        i = self.start + ind
        if i >= self.end:
            raise IndexError('segment object index out of range')
        return i

    def truncate(self, shift: int, how: str="both") -> 'BedEntry':
        '''
        how - ['left', 'rigth', 'both']
        '''
        s = self.start
        if how != "right":
            s += shift
        e = self.end 
        if how != 'left':
            e -= shift
        if s >= e:
            return BedEntry('', 0, 0, None)

        if self.peak is None or self.peak < s or self.peak > e:
            peak = None
        else:
            peak = self.peak
        return BedEntry(self.chr, s, e, peak) 
    
    def split(self, ind: int) -> tuple['BedEntry', 'BedEntry']:
        m = self[ind]
        s1, e1 = self.start, m
        s2, e2 = m, self.end

        if self.peak is None:
            p1, p2 = None, None
        else:
            if self.peak < m:
                p1, p2 = self.peak, None
            else:
                p1, p2 = None, self.peak

        return BedEntry(self.chr, s1, e1, p1), BedEntry(self.chr, s2, e2, p2)

    def expand(self, shift: int) -> 'BedEntry':
        st, end = self.start, self.end
        st -= shift
        end += shift
        return BedEntry(self.chr, st, end, self.peak)

    @classmethod
    def from_center(cls, chr: str, cntr: int, radius: int):
        st = cntr - radius
        end = cntr + 1 + radius
        return cls(chr, st, end, cntr)
        
        

In [8]:
@define
class BedData:
    entries : list[BedEntry] = field(repr=False, factory=list)
    executor : BedtoolsExecutor = BED_TOOLS_EXECUTOR
    sorted: bool = False

    def __len__(self) -> int:
        return len(self.entries)

    def __getitem__(self, ind) -> BedEntry:
        return self.entries[ind]

    @classmethod
    def from_file(cls, path: Union[Path, str], sort=False, presorted=False, header=False):
        if isinstance(path, str):
            path = Path(path)
        entries = []
        with path.open() as infile:
            if header:
                infile.readline()
            for line in infile:
                line = line.rstrip(END_LINE_CHARS)
                entry = BedEntry.from_line(line)
                entries.append(entry)
        self = cls(entries, sorted=sort)
        if sort and not presorted:
            self.sort()
        return self

    def sort(self):
        self.entries.sort()
        self.sorted = True

    def write(self, path:Union[Path, str], write_peak: bool=True):
        if isinstance(path, str):
            path = Path(path)
        with path.open("w") as output:
            for e in self.entries:
                line = e.to_line(include_peak=write_peak)
                print(line, file=output, sep="\t")

    def merge(self) -> 'BedData':
        store = temporary_file()
        self.write(store)
        out_path = temporary_file()
        out_path = self.executor.merge(store)
        return self.from_file(out_path, presorted=True)

    def subtract(self, other: 'BedData', full: bool) -> 'BedData':
        tmp1 = temporary_file()
        tmp2 = temporary_file()
        self.write(tmp1)
        other.write(tmp2)
        out_path = self.executor.subtract(tmp1, tmp2, full=full)
        return self.from_file(out_path, presorted=True)

    def closest(self, other: 'BedData', how: BedClosestMode) -> List[int]:
        '''
        for each feature in self distance to closest feature in other
        if no such feature exists - return None
        '''
        store1 = temporary_file()
        store2 = temporary_file()
        self.write(store1)
        other.write(store2)
        out_path = self.executor.closest(store1, store2, how=how)
        lst = []
        with open(out_path) as inp:
            fields = inp.readline().split()
            dist_ind = len(fields) - 1
            chrom_ind = len(fields) // 2
            inp.seek(0)
            for line in inp:
                fields = line.rstrip(END_LINE_CHARS).split()
                chrom = fields[chrom_ind]
                if chrom == ".":
                    lst.append(None)
                else:
                    dist = int(fields[dist_ind])
                    if dist < 0 and how is BedClosestMode.DOWNSTREAM:
                        lst.append(None)
                    else:
                        lst.append(dist)
        return lst

    def flank(self, genome: Union[str, Path, Genome], size: int) -> 'BedData':
        if isinstance(genome, Genome):
            genomesizes = temporary_file()
            genome.write_bed_genome_file(genomesizes)
            genome = genomesizes
        elif isinstance(genome, str):
            genome = Path(genome)
        elif not isinstance(genome, Path):
            raise Exception(f"Wrong type of genome argument: {type(genome)}")
        
        store = temporary_file()
        self.write(store)
        out_path = self.executor.flank(store, genome, size)
        return BedData.from_file(out_path)

    def append(self, e: BedEntry) -> None:
        self.entries.append(e)
        self.sorted = False

    def pop(self, ind) -> 'BedEntry':
        return self.entries.pop(ind)

    def size(self) -> int:
        return sum(len(e) for e in self.entries)

    def apply(self, fn : Callable[[BedEntry], Optional[BedEntry]]) -> 'BedData':
        new_entries = []
        for s in self.entries:
            s = fn(s)
            if s is not None:
                new_entries.append(s)
        return BedData(new_entries)
    
    def map(self, fn : Callable[[BedEntry], Optional[U]]) -> list[U]:
        lst = []
        for s in self.entries:
            s = fn(s)
            if s is not None:
                lst.append(s)
        return lst

    def filter(self, predicate: Callable[[BedEntry], bool]) -> 'BedData':
        flt_entries = []
        for s in self.entries:
            if predicate(s):
                flt_entries.append(s)
        return BedData(flt_entries)

    def global2local(self, global_ind: int) -> tuple[int, int]:
        ind = global_ind
        for si, e in enumerate(self.entries):
            if ind < len(e):
                break
            ind = ind - len(e)
        else: # no break
            raise IndexError("global segmentset index is out of range")
        return si, ind

    def retrieve(self, global_ind: int) -> int:
        si, ind = self.global2local(global_ind)
        return self.entries[si][ind]
    
    def sample(self, k: int) -> 'BedData':
        k = min(k, len(self))
        return BedData(random.sample(self.entries, k))

    def sample_shades(self, seg_size, k: int=1) -> list[BedEntry]:
        shift = seg_size // 2
        other = deepcopy(self)
        smpls = []
        for _ in range(k):        
            n = other.size()
            if n == 0:
                break

            g_ind = random.sample(range(n), k=1)[0]
            si, ind = other.global2local(g_ind)
            entry = other.pop(si)
            coord = entry[ind]
            
            new_entry = BedEntry.from_center(entry.chr, coord, shift)
            smpls.append(new_entry)
            
            en1, en2 = entry.split(ind)
            en1 = en1.truncate(shift, how='right')
            en2 = en2.truncate(shift, how="left")
            if len(en1) > 0:
                other.append(en1)
            if len(en2) > 0:
                other.append(en2)
        return smpls

In [9]:
def join_bed(beds: List[BedData],  sort=True) -> BedData:
    if any(not x.sorted for x in beds):
        entries = []
        for b in beds:
            entries.extend(b.entries)
        if sort:
            entries.sort()
        return BedData(entries, sorted=sort)
    entries = list(heapq.merge(*(b.entries for b in beds)))
    return BedData(entries, sorted=True)

In [10]:
import typing
from typing import List

@define
class UniqueTagger:
    parts: typing.Sequence[str]
    vocabularies: dict[str, list[str]] = field(repr=False)
    used_tags: set[str] = field(repr=False, factory=set) 
    max_occupancy: float = 0.5
    seed: int = 777
    max_size: int = field(init=False)
    random_generator: random.Random = field(init=False, repr=False)


    DEFAULT_PARTS: ClassVar[typing.Sequence[str]] = ("adj", "adj", "nat", "ani")
    DEFAULT_PARTS_PATH: ClassVar[dict[str, Path]] = {'adj': Path("adjectives.txt"), 'nat': Path('nations.txt'), 'ani': Path('animals.txt')}

    PARTS_FIELD: ClassVar[str]="PARTS"
    VOC_FIELD: ClassVar[str] = "VOC"
    TAGS_FIELD: ClassVar[str] = "USED_TAGS"
    PARTS_SEP: ClassVar[str] = '-'

    def __attrs_post_init__(self):
        self.max_size = self._calc_maxsize()
        self.random_generator = random.Random(self.seed)

    @classmethod
    def make(cls, parts: typing.Sequence[str], parts_path: dict[str, Path]):
        dt = {}
        for part in parts:
            with parts_path[part].open() as inp:
                voc =  [line.rstrip(END_LINE_CHARS) for line in inp]
                dt[part] = voc
        return cls(parts, dt)

    @classmethod
    def default(cls):
        return cls.make(cls.DEFAULT_PARTS, cls.DEFAULT_PARTS_PATH)        

    def write(self, path: Union[Path, str]):
        if isinstance(path, str):
            path = Path(path)
        with path.open('w') as store:
            dt = {self.PARTS_FIELD: self.parts,
                  self.VOC_FIELD: self.vocabularies,
                  self.TAGS_FIELD: list(self.used_tags)}
            json.dump(dt, store, indent=4)

    @classmethod
    def load(cls, path: Union[Path, str]) -> 'UniqueTagger':
        if isinstance(path, str):
            path = Path(path)
        with path.open() as inp:
            dt = json.load(inp)
        parts = dt[cls.PARTS_FIELD]
        if not isinstance(parts, list):
            raise Exception('Wrong format')
        vocs = dt[cls.VOC_FIELD]
        if not isinstance(vocs, dict):
            raise Exception('Wrong format')
        tags = dt[cls.TAGS_FIELD]
        if not isinstance(tags, list):
            raise Exception('Wrong format')

        return cls(tuple(parts), vocs, set(tags)) 
    
    def _non_unique_tag(self) -> str:
        tag_prts = [self.random_generator.choice(self.vocabularies[p]) for p in self.parts]
        tag = self.PARTS_SEP.join(tag_prts)
        return tag
    
    def tag(self) -> str:
        if len(self.used_tags) >= self.max_size:
            raise Exception("Max size for fast random generation reached")
        while (tag := self._non_unique_tag()) in self.used_tags:
            pass
        self.used_tags.add(tag)
        return tag
    
    def _calc_maxsize(self) -> int:
        cnt = 1
        for p in self.parts:
            cnt *= len(self.vocabularies[p])
        return int(cnt * self.max_occupancy) 

In [11]:
from threading import Lock
from contextlib import contextmanager
#https://gist.github.com/tylerneylon
class RWLock(object):
    """ RWLock class; this is meant to allow an object to be read from by
        multiple threads, but only written to by a single thread at a time. See:
        https://en.wikipedia.org/wiki/Readers%E2%80%93writer_lock
        Usage:
            my_obj_rwlock = RWLock()
            # When reading from my_obj:
            with my_obj_rwlock.r_locked():
                do_read_only_things_with(my_obj)
            # When writing to my_obj:
            with my_obj_rwlock.w_locked():
                mutate(my_obj)
    """

    def __init__(self):

        self.w_lock = Lock()
        self.num_r_lock = Lock()
        self.num_r = 0

    def r_acquire(self):
        self.num_r_lock.acquire()
        self.num_r += 1
        if self.num_r == 1:
            self.w_lock.acquire()
        self.num_r_lock.release()

    def r_release(self):
        assert self.num_r > 0
        self.num_r_lock.acquire()
        self.num_r -= 1
        if self.num_r == 0:
            self.w_lock.release()
        self.num_r_lock.release()

    @contextmanager
    def r_locked(self):
        try:
            self.r_acquire()
            yield
        finally:
            self.r_release()

    def w_acquire(self):
        self.w_lock.acquire()

    def w_release(self):
        self.w_lock.release()

    @contextmanager
    def w_locked(self):
        try:
            self.w_acquire()
            yield
        finally:
            self.w_release()

In [12]:
from threading import Lock

@define
class SeqDB:
    seq2tag: dict[str, str] = field(factory=dict)
    tag2seq: dict[str, str] = field(factory=dict)
    tagger: UniqueTagger = field(factory=UniqueTagger.default)
    lock: RWLock = RWLock() 
    
    def add(self, seq: Union[Sequence, str]) -> str:
        if isinstance(seq, Sequence):
            seq = seq.seq
        tag = self.get_tag(seq)
        if tag is None:
            with self.lock.w_locked(): 
                tag = self.tagger.tag()
                self.seq2tag[seq] = tag
                self.tag2seq[tag] = seq
        return tag

    def get_tag(self, seq: Union[Sequence, str]) -> Optional[str]:
        if isinstance(seq, Sequence):
            seq = seq.seq
        with self.lock.r_locked():
            tag = self.seq2tag.get(seq)
        return tag

    def get_seq(self, tag: str) -> Sequence:
        with self.lock.r_locked():
            seq = self.tag2seq[tag]
        return Sequence(seq)

In [13]:
from dataset import Dataset
from experiment import ExperimentType
from abc import abstractproperty, abstractmethod, ABCMeta
from math import ceil

class SubProtocol(metaclass=ABCMeta):

    @abstractproperty
    def data_type(self) -> ExperimentType:
        raise NotImplementedError()

    @abstractmethod
    def preprocess(self, cfgs: List[DatasetConfig]):
        '''
        prepare data given cgfs files of all datasets of specified datatype
        '''
        raise NotImplementedError()

    @abstractmethod
    def process(self, cfg: Union[List[DatasetConfig], DatasetConfig]) -> 'Dataset':
        '''
        create dataset from cfg file using it's data and data aquired during
        preprocess stage. Data from other cfg should not be used 
        '''
        raise NotImplementedError()

In [14]:
from exceptions import WrongDatasetTypeException
from typing import Iterator
from utils import undict


@define
class UDataset(metaclass=ABCMeta):
    name: str
    type: DatasetType
    tf_name: str
    entries: Optional[List[SeqEntry]] = field(repr=False)
    metainfo: dict
    disk_path: Path  

    SEQUENCE_FIELD: ClassVar[str] = "sequence"
    LABEL_FIELD: ClassVar[str] = "label"
    TAG_FIELD: ClassVar[str] = "tag"
    NO_INFO_VALUE: ClassVar[str] = "NoInfo"


    def closed(self) -> bool:
        return self.entries is None

    def __len__(self) -> int:
        if self.closed():
            raise Exception("Operation on closed dataset")
        return len(self.entries) # type: ignore

    def __iter__(self) -> Iterator[SeqEntry]:
        if self.closed():
            raise Exception("Operation on closed dataset")
        return iter(self.entries) # type: ignore

    def __getitem__(self, ind: int) -> SeqEntry:
        if self.closed():
            raise Exception("Operation on closed dataset")
        return self.entries[ind] # type: ignore
    
    def get_fields(self) -> list[str]:
        if self.type is DatasetType.TRAIN:
            return self.get_train_fields()
        if self.type is DatasetType.TEST:
            return self.get_test_fields()
        if self.type is DatasetType.VALIDATION:
            return self.get_valid_fields()
        if self.type is DatasetType.FULL:
            return self.get_full_fields()
        raise WrongDatasetTypeException(f"{self.type}")

    @abstractmethod
    def get_train_fields(self):
        raise NotImplementedError()
    
    @abstractmethod
    def get_test_fields(self):
        raise NotImplementedError()

    def get_valid_fields(self):
        return self.get_train_fields()
    
    def get_full_fields(self):
        return self.get_train_fields()

    def get(self, key, default):
        try:
            val = getattr(self, key)
        except AttributeError:
            val = self.metainfo.get(key, default)
        return val
    
    def to_tsv(self,
               path: Path,
               ds_fields: Optional[List[str]] = None):
        fields = self.get_fields()
        
        if ds_fields:
           ds_values = [self.get(fl, self.NO_INFO_VALUE) for fl in ds_fields]

        with open(path, "w") as out:
            if ds_fields:
                header = "\t".join(fields + ds_fields)
            else:
                header = "\t".join(fields)
            print(header, file=out)
            for en in self.entries:
                values = [en.get(fld, self.NO_INFO_VALUE) for fld in fields]  
                if ds_fields:
                    values.extend(ds_values) # type: ignore
                print("\t".join(values), file=out)
                    
    def to_json(self, path, mode: Optional[DatasetType] = DatasetType.TEST):
        raise NotImplementedError()

    @abstractmethod
    def to_canonical_format(self, path, mode: Optional[DatasetType] = DatasetType.TEST):
        raise NotImplementedError()

    def split(self):
        raise NotImplementedError()

    @classmethod
    def load(cls, path: Union[Path, str]) -> 'Dataset':
        raise NotImplementedError()
        

 name: str
    type: DatasetType
    tf_name: str
    entries: Optional[List[SeqEntry]] = field(repr=False)
    metainfo: dict
    disk_path: Path  

In [15]:
class ChIPSeqDataset(Dataset):
    def get_train_fields(self) -> list[str]:
        return [self.TAG_FIELD, self.SEQUENCE_FIELD,  self.LABEL_FIELD]
    
    def get_test_fields(self):
         return [self.TAG_FIELD, self.SEQUENCE_FIELD]
   
    def to_canonical_format(self, path):
        return self.to_tsv(path)

    @classmethod
    def load(cls, path: Union[Path, str], tp: DatasetType, fmt: str) -> 'ChIPSeqDataset':
        if isinstance(path, str):
            path = Path(path)
        # TODO: fix saving dataset (storing name, tf_name and metainfo in file)
        # TODO: add other fmts, maybe using class abstraction
        if fmt != ".tsv":
            raise NotImplementedError()
        
        self = cls(name="", tf_name="", entries=[], type=tp, metainfo={})
        fieldsname = self.get_fields()
        with path.open() as inp:
            _ = inp.readline() # header
            for line in inp:
                fields = line.split()
                dt = dict(zip(fieldsname, fields))
                entry = SeqEntry(**dt)
                self.entries.append(entry)
        return self

In [16]:
class NegativesSampler(metaclass=ABCMeta):
    @abstractmethod
    def sample(self, prot: 'ChIPSeqIrisProtocol', cfg: DatasetConfig) -> BedData:
        raise NotImplementedError

# TODO: move sampling functionality and args directly to samplers 
class ForeignPeakSampler(NegativesSampler):
    def sample(self, prot: 'ChIPSeqIrisProtocol', cfg: DatasetConfig) -> BedData:
        return prot.get_foreign_peaks(cfg)


class ShadesSampler(NegativesSampler):
    def sample(self, prot: 'ChIPSeqIrisProtocol', cfg: DatasetConfig) -> BedData:
        return prot.get_shades(cfg)

In [17]:
@define
class ChIPSeqIrisProtocol(SubProtocol):   
    mx_dist: int
    peak_size: int
    shades_per_peak: int
    negative_tf_ratio: int 
    genome: Genome = field(repr=False)
    root: Path
    db: SeqDB = field(factory=SeqDB)
    
    TF_MERGE_DIR_NAME: ClassVar[str] = "TF_MERGE"
    TF_FOREIGN_DIR_NAME: ClassVar[str] = "TF_FOREIGN"
    REAL_PEAKS_FILE_NAME: ClassVar[str] = "real.bed"
    ALL_PEAKS_FILE_NAME: ClassVar[str] = "all.bed"

    MX_DIST_FIELD: ClassVar[str] = "mx_dist"
    PEAK_SIZE_FIELD: ClassVar[str] = "peak_size"
    SHADES_PER_PEAK_FIELD: ClassVar[str] = "shades_per_peak"
    NEGATIVE_TF_RATIO_FIELD: ClassVar[str] = "negative_ratio"
    ROOT_FIELD: ClassVar[str] = "root_dir"
    GENOME_FIELD: ClassVar[str] = "genome"

    DEFAULT_ROOT = Path(".ChIPSeq")

    @property
    def data_type(self) -> ExperimentType:
        return ExperimentType.ChIPSeq

    @property
    def tf_merge_dir(self) -> Path:
        return self.root / self.TF_MERGE_DIR_NAME

    @property
    def tf_foreign_dir(self) -> Path:
        return self.root / self.TF_FOREIGN_DIR_NAME

    @property
    def real_peaks_path(self) -> Path:
        return self.root / self.REAL_PEAKS_FILE_NAME
   
    def mkdirs(self):
        self.root.mkdir(exist_ok=True, parents=True)
        self.tf_merge_dir.mkdir(exist_ok=True, parents=True)
        self.tf_foreign_dir.mkdir(exist_ok=True, parents=True)
    
    def __attrs_post_init__(self):
        if self.peak_size % 2 != 1:
            raise Exception(f"Peak size must be odd: {self.peak_size}")
        self.mkdirs()

    def tf_merge_path(self, tf_name:str) -> Path:
        return self.tf_merge_dir / f"{tf_name}.bed"

    def tf_foreign_path(self, tf_name:str) -> Path:
        return self.tf_foreign_dir / f"{tf_name}.bed"

    def merge_by_tf(self, cfgs: list[DatasetConfig], real_peaks: BedData):
        cfgs.sort(key=lambda x : x.tf)
        for tf_name, tf_records in groupby(cfgs, key=lambda x : x.tf):
            beds = [BedData.from_file(x.path, header=True) for x in tf_records]
            jnd = join_bed(beds)
            mrg = jnd.merge()
            mrg_path = self.tf_merge_path(tf_name)
            mrg.write(mrg_path, write_peak=False)

            foreign = real_peaks.subtract(mrg, full=True)
            foreign_path = self.tf_foreign_path(tf_name)
            foreign.write(foreign_path, write_peak=False)

    @staticmethod
    def map_peaks(bed: BedData) -> dict[int, BedData]:
        dt = defaultdict(BedData)
        for e in bed:
            dt[e.peak].append(BedEntry(e.chr, e.start, e.end))
        return dict(dt)

    def sample_shades(self, ds: BedData, tf: BedData) -> BedData:
        flanks = ds.flank(self.genome, self.mx_dist + self.peak_size // 2)
        sbstr = flanks.subtract(tf, full=False)
        dt = self.map_peaks(sbstr)
        for peak, sgmnts in dt.items():
            sgmnts = sgmnts.apply(lambda s: s.truncate(self.peak_size // 2)).filter(lambda s: len(s) > 0)
            dt[peak] = sgmnts
        
        entries = []
        for peak, sgmnts in dt.items():
            smpl = sgmnts.sample_shades(seg_size=self.peak_size, k=self.shades_per_peak)
            entries.extend(smpl)
        return BedData(entries)

    def get_shades(self, cfg: DatasetConfig) -> BedData:
        ds = BedData.from_file(cfg.path, header=True)
        tf_file = self.tf_merge_path(cfg.tf)
        tf = BedData.from_file(tf_file, header=True)
        return self.sample_shades(ds, tf)
    
    def get_foreign_peaks(self, cfg: DatasetConfig) -> BedData:
        ds = BedData.from_file(cfg.path, header=True)
        foreign_path = self.tf_foreign_path(cfg.tf)
        foreign = BedData.from_file(foreign_path)
        size = len(ds) * self.negative_tf_ratio
        return foreign.sample(size)
    
    def make_real_peaks(self, cfgs: List[DatasetConfig]) -> BedData:
        beds = [BedData.from_file(x.path, header=True) for x in configs]
        real_bed = join_bed(beds)
        real_bed.write(self.real_peaks_path)
        return real_bed

    def preprocess(self, cfgs: List[DatasetConfig]) -> None:
        real_bed = self.make_real_peaks(cfgs)
        self.merge_by_tf(cfgs, real_bed)
        
    @classmethod
    def from_dt(cls, dt: dict) -> 'ChIPSeqIrisProtocol':
        mx_dist=int(dt[cls.MX_DIST_FIELD])
        peak_size = int(dt[cls.PEAK_SIZE_FIELD])
        shades_per_peak = int(dt[cls.SHADES_PER_PEAK_FIELD])
        negative_tf_ratio = int(dt[cls.NEGATIVE_TF_RATIO_FIELD])
        genome = Genome.from_dir(dt[cls.GENOME_FIELD])

        root  = dt.get(cls.ROOT_FIELD, cls.DEFAULT_ROOT)
        root = Path(root)
    
        return cls(mx_dist=mx_dist,
                   peak_size=peak_size,
                   shades_per_peak=shades_per_peak,
                   negative_tf_ratio=negative_tf_ratio,
                   genome=genome,
                   root=root)

    @classmethod
    def from_json(cls, path: Union[Path, str]) -> 'ChIPSeqIrisProtocol':
        if isinstance(path, str):
            path = Path(path)
        with path.open() as inp:
            dt = json.load(inp)
        lower_dt = {key.lower() : value for key, value in dt.items() }
        return cls.from_dt(lower_dt)

    def get_positives(self, cfg: DatasetConfig) -> list[SeqEntry]:
        bed = BedData.from_file(cfg.path, header=True)
        seqs = self.genome.cut(bed)
        entries = [SeqEntry(s, 
                        BinaryLabel.POSITIVE,
                        self.db.add(s), 
                        {}) for s in seqs]
        return entries

    def get_negatives(self, cfg: DatasetConfig, sampler: NegativesSampler) -> List[SeqEntry]:
        bed = sampler.sample(self, cfg)
        seqs = self.genome.cut(bed)
        entries = [SeqEntry(s, 
                        BinaryLabel.NEGATIVE,
                        self.db.add(s), 
                        {}) for s in seqs]
        return entries

    def prepare_dataset(self, cfg: DatasetConfig, sampler: NegativesSampler, name: str):
        pos = self.get_positives(cfg)
        neg = self.get_negatives(cfg, sampler)
        return ChIPSeqDataset(name=name, 
                            tf_name=cfg.tf, 
                            type=cfg.ds_type, 
                            entries=pos+neg, 
                            metainfo=cfg.metainfo)

    @staticmethod
    def get_shades_ds_name(cfg: DatasetConfig) -> str:
        return f"{cfg.name}_shades"

    @staticmethod
    def get_foreign_ds_name(cfg: DatasetConfig) -> str:
        return f"{cfg.name}_foreign"

    def process(self, cfg: DatasetConfig) -> list[Dataset]:
        shades_ds = self.prepare_dataset(cfg,
                                         sampler=ShadesSampler(),
                                         name=self.get_shades_ds_name(cfg))
        foreign_ds = self.prepare_dataset(cfg,
                                          sampler=ForeignPeakSampler(),
                                          name=self.get_foreign_ds_name(cfg))
        return [shades_ds, foreign_ds]
    
     

In [18]:
def peakfls2configs(files: list[Path], 
                    name_parser= parse.compile("{tf_name}.{fl}@{exp_type}@{name}@Peaks.{unique_tag}.{dt_type}.peaks")) -> list[DatasetConfig]:
    all_records = []
    for val_fl in files:
        val_fl = val_fl.absolute()
        data = name_parser.parse(val_fl.name)
        if data is None or isinstance(data, parse.Match):
            raise Exception("Wrong peakfile name format")
        dt = {
            "name": data["name"],
            "exp_type": "ChIPSeq",
            "tf": data['tf_name'],
            "ds_type": data['dt_type'],
            "path": val_fl,
            "curation_status": "accepted",
            "protocol": "iris",
            "metainfo": {}
        }
        all_records.append(DatasetConfig.from_dict(dt))
    return all_records

In [19]:
val_files = [Path(x) for x in glob.glob(str(CHS_ROOT / '*.peaks'))]
configs = peakfls2configs(val_files)

In [20]:
IRIS_CHIPSEQ_CONFIG = {
    "mx_dist": 300,
    "peak_size": 301,
    "shades_per_peak": 1,
    "negative_ratio": 100,
    "genome": Path("/home_local/dpenzar/hg38")
}

In [31]:
protocol = ChIPSeqIrisProtocol.from_dt(IRIS_CHIPSEQ_CONFIG)

In [24]:
#protocol.preprocess(configs)

In [24]:
ds_lst = protocol.process(configs[0])

In [25]:
DS_DIR = Path('.ChIPSeqDatasets')

In [28]:
from pwmeval import PWMEvalPFMPredictor, PWMEvalPWMPredictor
from _benchmark import ModelEntry

In [23]:
OLD_BENCHMARK_CONFIG = Path("/home_local/dpenzar/test_config.json")

In [24]:
import json
with open(OLD_BENCHMARK_CONFIG) as inp:
    dt = json.load(inp)

In [25]:
dt['datasets'] = []

In [45]:
dt['scorers'] = dt['scorers'][0:3]

In [46]:
BENCHMARK_CONFIG = Path("/home_local/dpenzar/chipseq_config.json")
with BENCHMARK_CONFIG.open('w') as out:
    json.dump(dt, out, indent=4)

In [47]:
from benchmarkconfig import BenchmarkConfig

In [48]:
bench = BenchmarkConfig.from_json(BENCHMARK_CONFIG).make_benchmark()

In [49]:
MODELS_PATH = Path("/home_local/dpenzar/models.json")

In [50]:
with open(MODELS_PATH, "r") as inp:
    models = json.load(inp)
print(len(models))
for mcfg in models:
    tf_name = mcfg['tf']
    name = mcfg['name']
    path = mcfg["path"]
    bench.add_pfm(tf_name, name, path)

6664


In [51]:
# TODO : add add_dataset method to benchmark 

In [52]:
ds = ChIPSeqDataset.load('.ChIPSeqDatasets/THC_0258_foreign.bed', tp=DatasetType.TRAIN, fmt='.tsv')

In [57]:
AVAILABLE_PROCS = 20

In [58]:
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=AVAILABLE_PROCS)

In [63]:
from model import Model

In [70]:
ChIPSeqLOGS = Path("chipseq_logs")
ChIPSeqLOGS.mkdir(exist_ok=True, parents=True)

In [55]:
from concurrent.futures import as_completed

In [71]:
DS_DIR = Path(".ChIPSeqDatasets")

In [72]:
for cfg in configs:
    print(f"Dataset {cfg.name}")
    ds_path = DS_DIR/f"{protocol.get_shades_ds_name(cfg)}.bed"
    ds = ChIPSeqDataset.load(ds_path, tp=DatasetType.TRAIN, fmt='.tsv')
    ds.name = protocol.get_shades_ds_name(cfg)
    ds.tf_name = cfg.tf
    
    futures = {}
    for model in bench.models:
        #if not (isinstance(model, Model) or ds.tf_name is None or model.tfs is None or ds.tf_name in model.tfs):
        if not ds.tf_name in model.tfs:
            continue
        ft = executor.submit(bench.score_model_on_ds, model, ds)
        futures[ft] = f"{model.name}@{ds.name}"

    print(f"Submited {len(futures)} futures")

    for ft in as_completed(futures):
        name = futures[ft]
        path = ChIPSeqLOGS / f"{name}.json"
        try:
            scores = ft.result()
            with path.open("w") as outp:
                json.dump(scores, outp)
        except Exception as exc:
            print(exc)
            print(f"Error for {name}")
        else:
            print(f"Finished for {name}")

    

    ds_path = DS_DIR/f"{protocol.get_foreign_ds_name(cfg)}.bed"
    ds = ChIPSeqDataset.load(ds_path, tp=DatasetType.TRAIN, fmt='.tsv')
    ds.name = protocol.get_foreign_ds_name(cfg)
    ds.tf_name = cfg.tf

    futures = {}
    for model in bench.models:
        if not (isinstance(model, Model) or ds.tf_name is None or model.tfs is None or ds.tf_name in model.tfs):
            continue
        ft = executor.submit(bench.score_model_on_ds, model, ds)
        futures[ft] = f"{model.name}@{ds.name}"

    print(f"Submited {len(futures)} futures")
    for ft in as_completed(futures):
        name = futures[ft]
        path = ChIPSeqLOGS / f"{name}.json"

        try:
            scores = ft.result()
            with path.open("w") as outp:
                json.dump(scores, outp)
        except Exception as exc:
            print(f"Error for {name}")
            print(exc)
        else:
            print(f"Finished for {name}")
    


    

Dataset SI0159
Submited 0 futures
Submited 0 futures
Dataset THC_0074
Submited 0 futures
Submited 0 futures
Dataset THC_0246
Submited 1 futures
Finished for CHAMP1.FL@CHS@silly-champagne-burmese@HughesLab.Homer@Motif1.ppm_sumscore@THC_0246_shades
Submited 1 futures
Finished for CHAMP1.FL@CHS@silly-champagne-burmese@HughesLab.Homer@Motif1.ppm_sumscore@THC_0246_foreign
Dataset THC_0462.Rep-MICHELLE_0314
Submited 1 futures
Finished for MKX.FL@CHS@droopy-tomato-affenpinscher@HughesLab.Homer@Motif1.ppm_sumscore@THC_0462.Rep-MICHELLE_0314_shades
Submited 1 futures


KeyboardInterrupt: 

3363

In [24]:
sum([e.label.value for e in ds_lst[0].entries]) / len(ds_lst[0].entries)

NameError: name 'ds_lst' is not defined

In [44]:
d= BedData.from_file(configs[0].path, header=True)

In [33]:
len(ds_lst[1])

169882

In [34]:
from pwmeval import PWMEvalPWMPredictor

pfm_path = Path("/home_local/dpenzar/ibis-challenge/benchmark/example.pwm")
model = PWMEvalPWMPredictor.from_pfm(pfm_path, Path("/home_local/dpenzar/PWMEval/pwm_scoring"))


In [126]:
!ls /home_local/dpenzar/PWMEval/pwm_scoring

1.fasta  COPYING  Makefile     pwm_scoring.c  README.md   seqshuffle.c
a.txt	 log.txt  pwm_scoring  README	      seqshuffle


In [115]:
ds = prepare_dataset(protocol, configs[0], sampler=ForeignPeakSampler())

In [117]:
ds

ChIPSeqDataset(name='SI0159', type=<DatasetType.VALIDATION: 'val'>, tf_name='ZNF454', metainfo={})

In [21]:
protocol.preprocess(configs)

In [22]:
total_bed = BedData.from_file('.ChIPSEQ_DB/total.bed')

In [23]:
total_dataset = BedData()
for entry in total_bed:
    if len(entry) != PEAK_SIZE: 
        if entry.peak is not None:
            entry = BedEntry.from_center(entry.chr, entry.peak, radius=PEAK_SIZE//2)
        else:
            entry = BedEntry.from_center(entry.chr, (entry.start + entry.end) // 2, radius=PEAK_SIZE//2)
    total_dataset.append(entry)

In [24]:
total_seqs = genome.cut(total_dataset)